
# 🧭 Reddit Rule Violation: Research‑Backed EDA → Baseline Modeling Notebook

이 노트북은 **Kaggle "Jigsaw - Agile Community Rules Classification"** 과 유사한 과제 맥락에서, 제공된 `train.csv`(comment, rule 쌍) 기반으로 **연구 근거를 반영한 EDA**를 수행하고, **인사이트 → 특징 설계 → 베이스라인 학습**까지 **흐름 있게** 진행합니다.

## 참고/배경 (연구 & 레퍼런스)
- **Rule-based moderation with LLMs**: LLM이 서브레딧 규칙을 프롬프트로 받아 규칙 위반 여부를 추론하는 연구. 커뮤니티별 성능 편차가 존재하며 **룰 텍스트와 댓글 텍스트의 상호작용(쌍 입력)**이 핵심임.  
  - Kumar, AbuHashem, Durumeric (2023/2024): *Watch Your Language: Investigating Content Moderation with LLMs*. (ICWSM 2024)  
- **레딧 모더레이션 데이터셋/규범 연구**: 레딧에서 삭제/제거된 수백만 코멘트 분석, 커뮤니티 규범(macro/micro) 위반 탐지. **스팸/광고, 인신공격, 정치/규칙 위반** 등 이질적 규범 존재.  
  - Chandrasekharan & Gilbert (2018/2019): *Norm Violations*, *Hybrid Approaches*, *Crossmod* 등.
- **실무적 베이스라인**: 텍스트 분류에서 **TF‑IDF + 로지스틱 회귀**가 낮은 연산비로 강한 성능·AUC 확보. Toxic/abuse 탐지 벤치마크에서도 일관되게 강함.
- **스팸/광고 신호**: URL 수, 도메인, 반복문자, 과도한 대문자/구두점, 길이, 이모지 비율 등 **스타일 특징**이 유용함.

> 본 노트북은 위 근거를 반영해 **(1) 데이터 품질/분포 점검 → (2) 규칙/커뮤니티/텍스트 특성 분석 → (3) 룰-본문 쌍 상호작용 특징**(유사도, TF‑IDF 쌍 인코딩 등) → (4) **베이스라인 모델(AUC 검증)** → (5) 개선 로드맵 순으로 구성합니다.


In [ ]:

import os, re, math, string, unicodedata, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from scipy import sparse
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

DATA_PATH = '/mnt/data/train.csv' if os.path.exists('/mnt/data/train.csv') else 'train.csv'
df = pd.read_csv(DATA_PATH)
df.head()


## 1) 데이터 개요 / 무결성 점검

In [ ]:

print("Shape:", df.shape)
display(df.sample(5, random_state=42))
display(df.describe(include='all'))
print("\nNull counts:\n", df.isnull().sum())
print("\nDtypes:\n", df.dtypes)
dup_pairs = df.duplicated(subset=['body','rule']).sum()
print(f"Duplicated (body, rule) pairs: {dup_pairs}")


## 2) 타깃 분포 (`rule_violation`)

In [ ]:

target = 'rule_violation'
vc = df[target].value_counts().sort_index()
print(vc)
print("\nClass ratio:", (vc / vc.sum()).to_dict())

plt.figure(figsize=(4,3))
plt.bar(vc.index.astype(str), vc.values)
plt.title('Rule Violation Distribution')
plt.xlabel('rule_violation')
plt.ylabel('count')
plt.tight_layout()
plt.show()


## 3) Subreddit 분포 및 규칙 위반율

In [ ]:

sub_vc = df['subreddit'].value_counts()
print("Unique subreddits:", df['subreddit'].nunique())
display(sub_vc.head(20))

sub_stats = (df.groupby('subreddit')[target]
               .agg(['mean','count'])
               .rename(columns={'mean':'violation_rate','count':'n'})
               .sort_values('n', ascending=False))
display(sub_stats.head(20))

plt.figure(figsize=(7,6))
top_counts = sub_vc.head(20)
plt.barh(range(len(top_counts)), top_counts.values)
plt.yticks(range(len(top_counts)), top_counts.index)
plt.gca().invert_yaxis()
plt.title('Top 20 Subreddits (by count)')
plt.xlabel('count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,6))
top_subs = sub_stats.head(20)
plt.barh(range(len(top_subs)), top_subs['violation_rate'].values)
plt.yticks(range(len(top_subs)), top_subs.index)
plt.gca().invert_yaxis()
plt.title('Top 20 Subreddits - Violation Rate')
plt.xlabel('violation_rate')
plt.tight_layout()
plt.show()


## 4) Rule(규칙) 분포 & 위반율

In [ ]:

rule_vc = df['rule'].value_counts()
print("Unique rules:", df['rule'].nunique())
display(rule_vc.head(15))

rule_stats = (df.groupby('rule')[target]
                .agg(['mean','count'])
                .rename(columns={'mean':'violation_rate','count':'n'})
                .sort_values('n', ascending=False))
display(rule_stats.head(20))

plt.figure(figsize=(7,6))
top_rules = rule_vc.head(15)
plt.barh(range(len(top_rules)), top_rules.values)
plt.yticks(range(len(top_rules)), top_rules.index)
plt.gca().invert_yaxis()
plt.title('Top 15 Rules (by count)')
plt.xlabel('count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,6))
top_rules_stats = rule_stats.head(15)
plt.barh(range(len(top_rules_stats)), top_rules_stats['violation_rate'].values)
plt.yticks(range(len(top_rules_stats)), top_rules_stats.index)
plt.gca().invert_yaxis()
plt.title('Top 15 Rules - Violation Rate')
plt.xlabel('violation_rate')
plt.tight_layout()
plt.show()


## 5) 본문 길이/기초 통계 (스팸·규범 위반 관련 신호)

### 🔎 Lexicon 기반 특징이란?
Lexicon은 특정 주제(예: 욕설, 광고)와 관련된 **단어 사전**을 의미합니다.  
EDA에서 `url_cnt`, `upper_rt`, `exc_cnt` 같은 컬럼은 단어 사전이나 스타일 규칙에 기반해 생성된 특징입니다.  
즉, **본문 자체 의미보다 글쓰기 패턴(스타일)** 을 이용해 규칙 위반을 탐지하는 보조 신호입니다.


In [ ]:

def count_urls(text):
    return len(re.findall(r'https?://\S+|www\.\S+', str(text)))

def count_exclaims(text):
    return str(text).count('!')

def count_questions(text):
    return str(text).count('?')

def upper_ratio(text):
    s = str(text)
    letters = [c for c in s if c.isalpha()]
    if not letters:
        return 0.0
    upp = sum(1 for c in letters if c.isupper())
    return upp / len(letters)

def repeat_char_max(text):
    longest = 1
    last = ''
    cur = 0
    for ch in str(text):
        if ch == last:
            cur += 1
        else:
            longest = max(longest, cur)
            cur = 1
            last = ch
    longest = max(longest, cur)
    return longest

def emoji_ratio(text):
    s = str(text)
    total = len(s) if len(s)>0 else 1
    emojis = sum(1 for ch in s if ch in emoji_set)
    return emojis / total

emoji_set = set(list("😀😃😄😁😆😅😂🤣🥲😊🙂🙃😉😍😘😗😙😚😋😛😝😜🤪🤨🧐🤓😎🤩🥳😏😒😞😔😟😕🙁☹️😣😖😫😩😤😠😡🤬"))

df['body_len'] = df['body'].astype(str).str.len()
df['url_cnt'] = df['body'].apply(count_urls)
df['exc_cnt'] = df['body'].apply(count_exclaims)
df['q_cnt']   = df['body'].apply(count_questions)
df['upper_rt'] = df['body'].apply(upper_ratio)
df['rep_run']  = df['body'].apply(repeat_char_max)
df['emoji_rt'] = df['body'].apply(emoji_ratio)

display(df[['body_len','url_cnt','exc_cnt','q_cnt','upper_rt','rep_run','emoji_rt', 'rule_violation']].describe())

fig, axes = plt.subplots(2,3, figsize=(12,7)); axes = axes.ravel()
axes[0].hist(df['body_len'], bins=50); axes[0].set_title('body_len')
axes[1].hist(df['url_cnt'], bins=20);  axes[1].set_title('url_cnt')
axes[2].hist(df['upper_rt'], bins=30); axes[2].set_title('upper_rt')
axes[3].hist(df['exc_cnt'], bins=20);  axes[3].set_title('exc_cnt')
axes[4].hist(df['q_cnt'], bins=20);    axes[4].set_title('q_cnt')
axes[5].hist(df['rep_run'], bins=20);  axes[5].set_title('rep_run')
plt.tight_layout(); plt.show()

for m in ['body_len','url_cnt','upper_rt','exc_cnt','q_cnt','rep_run','emoji_rt']:
    plt.figure(figsize=(4,3))
    data0 = df.loc[df[target]==0, m].values
    data1 = df.loc[df[target]==1, m].values
    plt.boxplot([data0, data1], labels=['non-viol','viol'])
    plt.title(m)
    plt.tight_layout(); plt.show()

corr = df[['body_len','url_cnt','upper_rt','exc_cnt','q_cnt','rep_run','emoji_rt', target]].corr(numeric_only=True)[target].sort_values(ascending=False)
print("Correlation with target (pearson):\n", corr)


## 6) 경량 Lexicon 기반 신호 (욕설/광고)

In [ ]:

profanity = {'idiot','moron','stupid','dumb','retard','asshole','bastard'}
ad_words  = {'free','win','offer','discount','promo','sale','subscribe','click','visit','buy','deal','coupon','%off'}

def contains_any(text, vocab):
    s = str(text).lower()
    return int(any(tok in s for tok in vocab))

df['has_profanity'] = df['body'].apply(lambda x: contains_any(x, profanity))
df['has_adword']    = df['body'].apply(lambda x: contains_any(x, ad_words))

print(df[['has_profanity','has_adword', target]].groupby(['has_profanity','has_adword']).agg(['mean','count']))


## 7) 규칙 텍스트 vs 본문 상호작용 (유사도 특징)

### 📐 Jaccard 유사도
Jaccard 유사도는 **두 집합의 교집합 비율**로 정의됩니다.  
➡ 규칙 텍스트와 댓글 텍스트를 단어 집합으로 바꿔, 겹치는 단어 비율을 측정합니다.  

- 값 범위: 0 (겹침 없음) ~ 1 (완전히 동일)
- 규칙 위반 댓글일수록 규칙 문구와 단어가 겹칠 확률이 높아, Jaccard 값이 높아집니다.

👉 따라서 규칙 위반 탐지에서 **규칙-본문 상호작용**을 반영하는 중요한 신호입니다.


In [ ]:

def jaccard(a, b):
    sa, sb = set(a.split()), set(b.split())
    u = len(sa|sb); i = len(sa&sb)
    return i / u if u else 0.0

df['rule_body_jaccard'] = [jaccard(r, b) for r,b in zip(df['rule'].astype(str), df['body'].astype(str))]
print("rule_body_jaccard head:\n", df['rule_body_jaccard'].head())

plt.figure(figsize=(4,3))
plt.hist(df['rule_body_jaccard'], bins=30)
plt.title('Jaccard(rule, body)')
plt.tight_layout(); plt.show()

plt.figure(figsize=(4,3))
plt.boxplot([df.loc[df[target]==0, 'rule_body_jaccard'],
             df.loc[df[target]==1, 'rule_body_jaccard']], labels=['non-viol','viol'])
plt.title('Jaccard vs target')
plt.tight_layout(); plt.show()


## 8) 제공된 예시 텍스트(positive/negative example)와의 유사도/누설 점검

### 🚨 데이터 누설(Leakage) 점검
규칙 예시 텍스트(`positive_example_1` 등)가 댓글 본문에 포함되는 경우,  
모델은 '예시 문구=정답'이라는 편법을 학습해버릴 수 있습니다.  
따라서 `body_contains_xxx` 컬럼을 만들어 실제 포함 여부를 확인하고, 타깃과의 상관을 검토합니다.  
이는 **데이터셋 설계상 누설 여부를 검증하는 핵심 절차**입니다.


In [ ]:

ex_cols = ['positive_example_1','positive_example_2','negative_example_1','negative_example_2']
for col in ex_cols:
    df[f'body_contains_{col}'] = df.apply(lambda r: int(str(r[col])[:120].lower() in str(r['body']).lower()), axis=1)
    print(col, df[f'body_contains_{col}'].sum())

leak_cols = [c for c in df.columns if c.startswith('body_contains_')]
display(df[leak_cols + [target]].groupby(leak_cols)[target].agg(['mean','count']).sort_values('count', ascending=False).head(10))


## 9) 모델링 입력 구성 (텍스트 + 스타일 + 상호작용)

In [ ]:

df['rule_sep_body'] = df['rule'].astype(str) + ' [SEP] ' + df['body'].astype(str)
num_cols = ['body_len','url_cnt','exc_cnt','q_cnt','upper_rt','rep_run','emoji_rt','has_profanity','has_adword','rule_body_jaccard']
X_text = df['rule_sep_body']; X_num = df[num_cols]
y = df[target].astype(int)
display(X_num.head())


## 10) 베이스라인: TF‑IDF(word+char) + 로지스틱 회귀 (Stratified 5‑Fold AUC)

In [ ]:

from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

tf_word = TfidfVectorizer(strip_accents='unicode', lowercase=True, ngram_range=(1,2), min_df=2, max_features=40000)
tf_char = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=2, max_features=60000)
scaler  = StandardScaler(with_mean=False)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_pred = np.zeros(len(df), dtype=float)
fold_scores = []

for fold, (tr, va) in enumerate(skf.split(X_text, y), 1):
    Xw_tr = tf_word.fit_transform(X_text.iloc[tr]); Xw_va = tf_word.transform(X_text.iloc[va])
    Xc_tr = tf_char.fit_transform(X_text.iloc[tr]); Xc_va = tf_char.transform(X_text.iloc[va])
    Xn_tr = scaler.fit_transform(X_num.iloc[tr]);   Xn_va = scaler.transform(X_num.iloc[va])
    X_tr = hstack([Xw_tr, Xc_tr, Xn_tr], format='csr')
    X_va = hstack([Xw_va, Xc_va, Xn_va], format='csr')
    clf = LogisticRegression(solver='saga', max_iter=3000, n_jobs=-1, class_weight='balanced', C=2.0, penalty='l2')
    clf.fit(X_tr, y.iloc[tr])
    oof_pred[va] = clf.predict_proba(X_va)[:,1]
    auc = roc_auc_score(y.iloc[va], oof_pred[va])
    fold_scores.append(auc)
    print(f"[Fold {fold}] AUC = {auc:.4f}")
print("\nCV AUC:", np.mean(fold_scores).round(4), "+/-", np.std(fold_scores).round(4))


## 11) 단어 중요도 살펴보기 (참고)

In [ ]:

try:
    word_feats = np.array(tf_word.get_feature_names_out())
    coefs = clf.coef_[0][:len(word_feats)]
    idx_top = np.argsort(-coefs)[:30]
    idx_bot = np.argsort(coefs)[:30]
    print("Top +coef tokens (push to violation):")
    print(list(zip(word_feats[idx_top], coefs[idx_top])))
    print("\nTop -coef tokens (push to non-violation):")
    print(list(zip(word_feats[idx_bot], coefs[idx_bot])))
except Exception as e:
    print("Feature importance preview skipped:", e)



## 12) 주요 인사이트 요약 → 액션 아이템

- **서브레딧·규칙별 분포 차이**: 데이터는 커뮤니티/룰에 따라 **표본 수와 위반율**이 상이합니다. **Stratified split** 및 **커뮤니티/룰 분포 보정**이 필요합니다.
- **스팸/광고 신호**: URL 수, 반복문자, 대문자 비율, 감탄/물음표 과다 등 **스타일 특징**이 위반과 상관(상세 수치 위 셀 참고).
- **룰-본문 상호작용**: `rule` 텍스트와 `body`간 **토큰 공유율(Jaccard)** 또는 **합쳐서 벡터화(rule [SEP] body)**가 유효한 신호로 보입니다.
- **예시 텍스트 누설 점검**: 댓글 본문이 예시 문자열 일부를 포함하는 사례가 있는지 반드시 확인(상기 결과 참고). 존재 시, 해당 특징 사용은 금지하거나 교정 필요.



## 13) 전체 재학습 & 제출 함수 (test.csv 가정)

### 📐 Jaccard 유사도
Jaccard 유사도는 **두 집합의 교집합 비율**로 정의됩니다.  
➡ 규칙 텍스트와 댓글 텍스트를 단어 집합으로 바꿔, 겹치는 단어 비율을 측정합니다.  

- 값 범위: 0 (겹침 없음) ~ 1 (완전히 동일)
- 규칙 위반 댓글일수록 규칙 문구와 단어가 겹칠 확률이 높아, Jaccard 값이 높아집니다.

👉 따라서 규칙 위반 탐지에서 **규칙-본문 상호작용**을 반영하는 중요한 신호입니다.


In [ ]:

def jaccard(a, b):
    sa, sb = set(a.split()), set(b.split())
    u = len(sa|sb); i = len(sa&sb)
    return i / u if u else 0.0

def fit_full_and_predict(train_df, test_df):
    X_text_tr = train_df['rule'].astype(str) + ' [SEP] ' + train_df['body'].astype(str)
    X_text_te = test_df['rule'].astype(str) + ' [SEP] ' + test_df['body'].astype(str)
    y_tr = train_df['rule_violation'].astype(int)

    # 파생 생성 for test
    def enrich(df_):
        df_ = df_.copy()
        df_['body_len'] = df_['body'].astype(str).str.len()
        df_['url_cnt'] = df_['body'].apply(lambda t: len(re.findall(r'https?://\S+|www\.\S+', str(t))))
        df_['exc_cnt'] = df_['body'].apply(lambda t: str(t).count('!'))
        df_['q_cnt']   = df_['body'].apply(lambda t: str(t).count('?'))
        def upper_ratio(text):
            s = str(text)
            letters = [c for c in s if c.isalpha()]
            if not letters:
                return 0.0
            upp = sum(1 for c in letters if c.isupper())
            return upp / len(letters)
        df_['upper_rt'] = df_['body'].apply(upper_ratio)
        def repeat_char_max(text):
            longest = 1; last=''; cur=0
            for ch in str(text):
                if ch == last: cur += 1
                else: longest = max(longest, cur); cur=1; last=ch
            longest = max(longest, cur)
            return longest
        df_['rep_run']  = df_['body'].apply(repeat_char_max)
        emoji_set = set(list("😀😃😄😁😆😅😂🤣🥲😊🙂🙃😉😍😘😗😙😚😋😛😝😜🤪🤨🧐🤓😎🤩🥳😏😒😞😔😟😕🙁☹️😣😖😫😩😤😠😡🤬"))
        df_['emoji_rt'] = df_['body'].apply(lambda s: (sum(1 for ch in str(s) if ch in emoji_set) / (len(str(s)) if len(str(s))>0 else 1)))
        profanity = {'idiot','moron','stupid','dumb','retard','asshole','bastard'}
        ad_words  = {'free','win','offer','discount','promo','sale','subscribe','click','visit','buy','deal','coupon','%off'}
        df_['has_profanity'] = df_['body'].apply(lambda x: int(any(tok in str(x).lower() for tok in profanity)))
        df_['has_adword']    = df_['body'].apply(lambda x: int(any(tok in str(x).lower() for tok in ad_words)))
        df_['rule_body_jaccard'] = [jaccard(r, b) for r,b in zip(df_['rule'].astype(str), df_['body'].astype(str))]
        return df_

    train_df = enrich(train_df); test_df = enrich(test_df)

    num_cols = ['body_len','url_cnt','exc_cnt','q_cnt','upper_rt','rep_run','emoji_rt','has_profanity','has_adword','rule_body_jaccard']
    X_num_tr = train_df[num_cols]; X_num_te = test_df[num_cols]

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from scipy.sparse import hstack

    tf_word = TfidfVectorizer(strip_accents='unicode', lowercase=True, ngram_range=(1,2), min_df=2, max_features=40000)
    tf_char = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=2, max_features=60000)
    scaler  = StandardScaler(with_mean=False)

    Xw_tr = tf_word.fit_transform(X_text_tr); Xw_te = tf_word.transform(X_text_te)
    Xc_tr = tf_char.fit_transform(X_text_tr); Xc_te = tf_char.transform(X_text_te)
    Xn_tr = scaler.fit_transform(X_num_tr);   Xn_te = scaler.transform(X_num_te)

    X_tr = hstack([Xw_tr, Xc_tr, Xn_tr], format='csr')
    X_te = hstack([Xw_te, Xc_te, Xn_te], format='csr')

    clf = LogisticRegression(solver='saga', max_iter=3000, n_jobs=-1, class_weight='balanced', C=2.0, penalty='l2')
    clf.fit(X_tr, y_tr)
    proba = clf.predict_proba(X_te)[:,1]
    return proba

def make_submission(test_csv_path, out_path='submission.csv'):
    test_df = pd.read_csv(test_csv_path)
    probs = fit_full_and_predict(df.copy(), test_df.copy())
    sub = pd.DataFrame({'row_id': test_df['row_id'], 'rule_violation': probs})
    sub.to_csv(out_path, index=False)
    print("Saved:", out_path)
    return sub

print("Ready: call make_submission('test.csv') when test is available.")


In [ ]:
make_submission('test.csv')